#### 先从MySQL导入数据到DataFrame：

In [1]:
import pymysql.cursors
import pandas as pd

db = pymysql.connect(host='localhost',user='root',password='root',db='python',charset='utf8',cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()
sql = 'SELECT * FROM car_loss;'
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(list(result))
db.close()

df

,EngSize,Age,Gender,Marital,exp,Owner,vAge,Garage,AntiTFD,import,Loss
0,2.0,56,男,,20,公司,10,有,有防盗装置,进口,0.0
1,1.8,41,男,,20,公司,9,有,无防盗装置,国产,0.0
2,2.0,44,男,,20,公司,8,有,有防盗装置,国产,0.0
3,1.6,56,男,,20,公司,7,有,有防盗装置,国产,0.0
4,1.8,45,男,,20,公司,7,无,无防盗装置,国产,0.0
...,...,...,...,...,...,...,...,...,...,...,...
16927,1.8,22,女,未婚,0,私人,1,有,有防盗装置,国产,976.0
16928,2.5,22,男,未婚,0,私人,1,有,无防盗装置,进口,855.6
16929,1.8,21,男,未婚,0,私人,1,有,无防盗装置,国产,0.0
16930,1.8,21,女,未婚,0,私人,1,有,无防盗装置,进口,3328.0


## 一、处理空字符串""

说明：由于空字符串""无法被检测为空值，且不方便后续填补，故需要先检测各字段是否存在空字符串。

### 1、检测各字段是否含有空字符串""

#### 检测某字段是否存在空字符串

In [136]:
(df['Gender'] == '').any()

True

#### 检测各字段是否含有空字符串

In [139]:
(df == '').any()

EngSize    False
Age        False
Gender      True
Marital     True
exp        False
Owner       True
vAge       False
Garage      True
AntiTFD     True
import      True
Loss       False
dtype: bool

#### 检测所有字段是否含有空字符串

In [140]:
(df == '').values.any()

True

#### 统计某字段为空字符串的数量

In [101]:
df['Gender'] == ''

0       False
1       False
2       False
3       False
4       False
        ...  
4228    False
4229    False
4230    False
4231    False
4232    False
Name: Gender, Length: 4233, dtype: bool

In [102]:
(df['Gender'] == '').sum()

5

#### 统计各字段存在空字符串总量

值得注意的是，在导入空字符串的时候，数值型字段会存储为0，字符型字段才会以空字符串的形式存储。

In [103]:
(df == '').sum()

C:\Users\romak\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


EngSize    0
Age        0
Gender     5
Marital    5
exp        0
Owner      5
vAge       0
Garage     5
AntiTFD    5
import     5
Loss       0
dtype: int64

### 2、检测数值型字段是否存在存储为 0 的空字符串

需要注意的是：数值型字段本身就有可能是0，并非由导入空字符串造成。

In [104]:
(df == 0).sum()

EngSize       5
Age           5
Gender        0
Marital       0
exp         172
Owner         0
vAge          5
Garage        0
AntiTFD       0
import        0
Loss       3028
dtype: int64

### 3、把空字符串""填补成空值NaN

#### 把指定列的空字符串""填补成空值NaN

In [105]:
import numpy as np
df['Gender'] = df['Gender'].map(lambda x: np.nan if x == '' else x)

In [106]:
df['Gender'].isna().sum()

5

In [107]:
df.isna().sum()

EngSize    0
Age        0
Gender     5
Marital    0
exp        0
Owner      0
vAge       0
Garage     0
AntiTFD    0
import     0
Loss       0
dtype: int64

#### 把所有字段的空字符串""填补成空值NaN

In [108]:
df = df.applymap(lambda x: np.nan if x == '' else x)
df

,EngSize,Age,Gender,Marital,exp,Owner,vAge,Garage,AntiTFD,import,Loss
0,2.0,56,男,NaN,20,公司,10,有,有防盗装置,进口,0.0
1,1.8,41,男,NaN,20,公司,9,有,无防盗装置,国产,0.0
2,2.0,44,男,NaN,20,公司,8,有,有防盗装置,国产,0.0
3,1.6,56,男,NaN,20,公司,7,有,有防盗装置,国产,0.0
4,1.8,45,男,NaN,20,公司,7,无,无防盗装置,国产,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4228,1.8,22,女,未婚,0,私人,1,有,有防盗装置,国产,976.0
4229,2.5,22,男,未婚,0,私人,1,有,无防盗装置,进口,855.6
4230,1.8,21,男,未婚,0,私人,1,有,无防盗装置,国产,0.0
4231,1.8,21,女,未婚,0,私人,1,有,无防盗装置,进口,3328.0


In [109]:
df.isna().sum()

EngSize    0
Age        0
Gender     5
Marital    5
exp        0
Owner      5
vAge       0
Garage     5
AntiTFD    5
import     5
Loss       0
dtype: int64

## 二、处理空值NaN

### 1、检测各字段是否存在空值NaN

#### 检测所有字段是否存在空值NaN

In [110]:
df.isna().values.any()

True

#### 检测各字段是否存在空值NaN

In [111]:
df.isna().any()

EngSize    False
Age        False
Gender      True
Marital     True
exp        False
Owner       True
vAge       False
Garage      True
AntiTFD     True
import      True
Loss       False
dtype: bool

### 2、统计各字段空值NaN情况

#### 统计所有字段存在空值NaN总量

In [112]:
df.isna().values.sum()

30

#### 统计各字段存在空值NaN的数量

In [113]:
df.isna().sum()

EngSize    0
Age        0
Gender     5
Marital    5
exp        0
Owner      5
vAge       0
Garage     5
AntiTFD    5
import     5
Loss       0
dtype: int64

#### 统计各字段非空值的数量

In [6]:
df.notna().sum()

EngSize    16932
Age        16932
Gender     16932
Marital    16932
exp        16932
Owner      16932
vAge       16932
Garage     16932
AntiTFD    16932
import     16932
Loss       16932
dtype: int64

#### 统计各字段空值NaN占比

In [114]:
df.count()

EngSize    4233
Age        4233
Gender     4228
Marital    4228
exp        4233
Owner      4228
vAge       4233
Garage     4228
AntiTFD    4228
import     4228
Loss       4233
dtype: int64

In [115]:
df.isna().sum()/(df.count()+df.isna().sum())

EngSize    0.000000
Age        0.000000
Gender     0.001183
Marital    0.001183
exp        0.000000
Owner      0.001183
vAge       0.000000
Garage     0.001183
AntiTFD    0.001183
import     0.001183
Loss       0.000000
dtype: float64

### 3、舍弃空值NaN

In [116]:
df

,EngSize,Age,Gender,Marital,exp,Owner,vAge,Garage,AntiTFD,import,Loss
0,2.0,56,男,NaN,20,公司,10,有,有防盗装置,进口,0.0
1,1.8,41,男,NaN,20,公司,9,有,无防盗装置,国产,0.0
2,2.0,44,男,NaN,20,公司,8,有,有防盗装置,国产,0.0
3,1.6,56,男,NaN,20,公司,7,有,有防盗装置,国产,0.0
4,1.8,45,男,NaN,20,公司,7,无,无防盗装置,国产,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4228,1.8,22,女,未婚,0,私人,1,有,有防盗装置,国产,976.0
4229,2.5,22,男,未婚,0,私人,1,有,无防盗装置,进口,855.6
4230,1.8,21,男,未婚,0,私人,1,有,无防盗装置,国产,0.0
4231,1.8,21,女,未婚,0,私人,1,有,无防盗装置,进口,3328.0


#### 舍弃整列为空值NaN的列

舍弃行的时候，删除axis参数即可。

In [117]:
df['kongzhi'] = np.nan
df

,EngSize,Age,Gender,Marital,exp,Owner,vAge,Garage,AntiTFD,import,Loss,kongzhi
0,2.0,56,男,NaN,20,公司,10,有,有防盗装置,进口,0.0,NaN
1,1.8,41,男,NaN,20,公司,9,有,无防盗装置,国产,0.0,NaN
2,2.0,44,男,NaN,20,公司,8,有,有防盗装置,国产,0.0,NaN
3,1.6,56,男,NaN,20,公司,7,有,有防盗装置,国产,0.0,NaN
4,1.8,45,男,NaN,20,公司,7,无,无防盗装置,国产,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4228,1.8,22,女,未婚,0,私人,1,有,有防盗装置,国产,976.0,NaN
4229,2.5,22,男,未婚,0,私人,1,有,无防盗装置,进口,855.6,NaN
4230,1.8,21,男,未婚,0,私人,1,有,无防盗装置,国产,0.0,NaN
4231,1.8,21,女,未婚,0,私人,1,有,无防盗装置,进口,3328.0,NaN


In [118]:
df.dropna(axis=1,how='all',inplace=True)
df

,EngSize,Age,Gender,Marital,exp,Owner,vAge,Garage,AntiTFD,import,Loss
0,2.0,56,男,NaN,20,公司,10,有,有防盗装置,进口,0.0
1,1.8,41,男,NaN,20,公司,9,有,无防盗装置,国产,0.0
2,2.0,44,男,NaN,20,公司,8,有,有防盗装置,国产,0.0
3,1.6,56,男,NaN,20,公司,7,有,有防盗装置,国产,0.0
4,1.8,45,男,NaN,20,公司,7,无,无防盗装置,国产,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4228,1.8,22,女,未婚,0,私人,1,有,有防盗装置,国产,976.0
4229,2.5,22,男,未婚,0,私人,1,有,无防盗装置,进口,855.6
4230,1.8,21,男,未婚,0,私人,1,有,无防盗装置,国产,0.0
4231,1.8,21,女,未婚,0,私人,1,有,无防盗装置,进口,3328.0


#### 舍弃空值NaN数量 ≥ 10个的字段

舍弃行的时候，删除axis参数即可。

In [119]:
df.dropna(axis=1,thresh=10,inplace=True)
df

,EngSize,Age,Gender,Marital,exp,Owner,vAge,Garage,AntiTFD,import,Loss
0,2.0,56,男,NaN,20,公司,10,有,有防盗装置,进口,0.0
1,1.8,41,男,NaN,20,公司,9,有,无防盗装置,国产,0.0
2,2.0,44,男,NaN,20,公司,8,有,有防盗装置,国产,0.0
3,1.6,56,男,NaN,20,公司,7,有,有防盗装置,国产,0.0
4,1.8,45,男,NaN,20,公司,7,无,无防盗装置,国产,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4228,1.8,22,女,未婚,0,私人,1,有,有防盗装置,国产,976.0
4229,2.5,22,男,未婚,0,私人,1,有,无防盗装置,进口,855.6
4230,1.8,21,男,未婚,0,私人,1,有,无防盗装置,国产,0.0
4231,1.8,21,女,未婚,0,私人,1,有,无防盗装置,进口,3328.0


#### 只要存在空值NaN的字段都舍弃

舍弃行的时候，删除axis参数即可。

In [120]:
df.dropna(axis=1,inplace=False)    # inplace设置为True才会真实生效

,EngSize,Age,exp,vAge,Loss
0,2.0,56,20,10,0.0
1,1.8,41,20,9,0.0
2,2.0,44,20,8,0.0
3,1.6,56,20,7,0.0
4,1.8,45,20,7,0.0
...,...,...,...,...,...
4228,1.8,22,0,1,976.0
4229,2.5,22,0,1,855.6
4230,1.8,21,0,1,0.0
4231,1.8,21,0,1,3328.0


### 4、填补空值NaN

In [121]:
import pandas as pd
import numpy  as np
df = pd.DataFrame([\
                   ['frank', 'M',    np.nan,np.nan], \
                   [np.nan , np.nan, np.nan,np.nan], \
                   ['tom'  , 'M',    35,np.nan], \
                   ['ted'  , 'M',    33,np.nan], \
                   ['jean' , np.nan, 21,np.nan], \
                   ['lisa' , 'F',    20,np.nan]])
df.columns = ['name', 'gender', 'age','city']
df

,name,gender,age,city
0,frank,M,NaN,NaN
1,NaN,NaN,NaN,NaN
2,tom,M,35.0,NaN
3,ted,M,33.0,NaN
4,jean,NaN,21.0,NaN
5,lisa,F,20.0,NaN


#### 所有字段的空值NaN填补为0

In [122]:
df.fillna(0,inplace=False)    # 要想生效，把inplace改为True即可。

,name,gender,age,city
0,frank,M,0.0,0.0
1,0,0,0.0,0.0
2,tom,M,35.0,0.0
3,ted,M,33.0,0.0
4,jean,0,21.0,0.0
5,lisa,F,20.0,0.0


#### 某字段的空值NaN填补为0

In [123]:
df['name'].fillna(0,inplace=False)    # 要想生效，把inplace改为True即可。

0    frank
1        0
2      tom
3      ted
4     jean
5     lisa
Name: name, dtype: object

#### 用平均值填补空值NaN

In [125]:
df['age'].fillna(df['age'].mean(),inplace=False)    # 要想生效，把inplace改为True即可。

0    27.25
1    27.25
2    35.00
3    33.00
4    21.00
5    20.00
Name: age, dtype: float64

#### 根据性别的平均值填补空值NaN

In [126]:
df2 = df.groupby('gender')['age'].transform('mean')
df2

0    34.0
1     NaN
2    34.0
3    34.0
4     NaN
5    20.0
Name: age, dtype: float64

In [127]:
?df.groupby

In [70]:
?df.transform

In [128]:
df['age'].fillna(df2,inplace=False)    # 要想生效，把inplace改为True即可。

0    34.0
1     NaN
2    35.0
3    33.0
4    21.0
5    20.0
Name: age, dtype: float64

#### 用众数填补空值NaN

In [129]:
mode = df['gender'].mode()[0]
mode

'M'

In [130]:
df['gender'].fillna(mode,inplace=False)

0    M
1    M
2    M
3    M
4    M
5    F
Name: gender, dtype: object

#### 用前一个非空值填补空值NaN

In [131]:
df['age'].fillna(method='ffill',inplace=False)    # 要想生效，把inplace改为True即可。

0     NaN
1     NaN
2    35.0
3    33.0
4    21.0
5    20.0
Name: age, dtype: float64

#### 用后一个有效值填补空值NaN

In [132]:
df['age'].fillna(method='bfill',inplace=False)    # 要想生效，把inplace改为True即可。

0    35.0
1    35.0
2    35.0
3    33.0
4    21.0
5    20.0
Name: age, dtype: float64

#### 使用内插法填补空值NaN

In [133]:
df['gender'].interpolate()

0      M
1    NaN
2      M
3      M
4    NaN
5      F
Name: gender, dtype: object

In [86]:
?df.interpolate